
# Part 1 - Hand Calculated Graph Centrality

In this part, we will load the Kite network data and perform a graph centrality algorithm by hand. First we load the data again:

In [4]:
# Hide some silly output
import logging
logging.getLogger("requests").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)

# Import everything we need
import graphlab as gl

# Load Data
kite_vertices = gl.SFrame.read_csv('../Week1/kite_vertices.csv')
kite_edges = gl.SFrame.read_csv('../Week1/kite_edges.csv')

# Create graph
g_kite = gl.SGraph()
g_kite = g_kite.add_vertices(vertices=kite_vertices, vid_field='name')
g_kite = g_kite.add_edges(edges=kite_edges, src_field='src', dst_field='dst')

# Visualize graph?
gl.canvas.set_target('ipynb')
g_kite.show(vlabel="id")

PROGRESS: Finished parsing file /home/james/Development/Masters/IndependentStudy/Week1/kite_vertices.csv
PROGRESS: Parsing completed. Parsed 10 lines in 0.018072 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /home/james/Development/Masters/IndependentStudy/Week1/kite_vertices.csv
PROGRESS: Parsing completed. Parsed 10 lines in 0.017847 secs.
PROGRESS: Finished parsing file /home/james/Development/Masters/IndependentStudy/Week1/kite_edges.csv
PROGRESS: Parsing completed. Parsed 18 lines in 0.019082 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[str,str,str]
If parsing fails due to incorrect types, you can

<IPython.core.display.Javascript object>

# Part 2 - Crawling Social Data

In this part, I will use the Twitter / Facebook / LinkedIn API to download a graphical data of my portion of the social graph, and attempt to visualize it in Gephi or Neo4J.

## TODO

- Download facebook python api
- Setup API keys for personal account
- Query FB for my friends (and maybe friends of friends) and their data
- Display as a graph somehow
  - Show ages of friends in different colors?
  - With friends of friends, can I detect clusters?

# Part 3 - Textual Analysis of Tweets from Political ThinkTanks

In this part, I will download the tweet streams from different political 'think tanks' and perform a simple frequency analysis to see if there are any insights we can derive about the political leanings of these institutions. As an example of what tweet strams I will parse:

- https://twitter.com/fairmediawatch - Fairness and Accuracy In Reporting
- https://twitter.com/AccuracyInMedia - Accuracy In Media
- https://twitter.com/ips_dc - Institute for Policy Studies
- https://twitter.com/heritage - Heritage Foundation